In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:
df_old = pd.read_pickle('drive/MyDrive/benchmark/question_new.pkl')
df_new = pd.read_pickle('drive/MyDrive/benchmark/question_old.pkl')

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=bfd773770293082e8463d9341be18ee8c980a89ab2a97fca55ab1b7254d1cc2b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("pszemraj/pegasus-x-large-book-summary")

model = AutoModelForSeq2SeqLM.from_pretrained(
    "pszemraj/pegasus-x-large-book-summary",
)


In [ ]:
import torch
from transformers import pipeline

summarizer = pipeline('summarization',
                      model=model,
                      tokenizer=tokenizer,
                      device= 0 if torch.cuda.is_available() else -1,
                      )

In [ ]:
import re
def summarize(text):
  text = re.sub(r"\((.*?)\)", "", text)
  params = {
    "max_length": int(len(text.split())*0.6),
    "min_length": 10,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "repetition_penalty": 3.5,
    "length_penalty": 0.2,
    "encoder_no_repeat_ngram_size": 3,
    "num_beams": 1,
  }
  result = summarizer(text, **params)
  return result[0]['summary_text']

In [ ]:
df_new['10_beam_summary'] = df_new['wiki'].apply(summarize)
df_old['10_beam_summary'] = df_old['wiki'].apply(summarize)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
df_new.to_pickle('drive/MyDrive/benchmark/df_new_10_beam.pkl')
df_old.to_pickle('drive/MyDrive/benchmark/df_old_10_beam.pkl')

In [ ]:
for i in range(len(df)):
  df['wiki'][i] = df['wiki'][i].replace('You can help Wikipedia by expanding it.', '')
  df['wiki'][i] = df['wiki'][i].replace('This pharmacology-related article is a stub.', '')
  if 'The requested page title contains unsupported' in df['wiki'][i] or len(df['wiki'][i].split()) < 20:
    df = df.drop(i)

In [ ]:
df = df.reset_index()

In [ ]:
import random
df

,level_0,index,cmpdname,wiki
0,0,0,Acetylcarnitine,"Acetyl-L-carnitine, ALCAR or ALC, is an acetyl..."
1,1,1,1-Aminopropan-2-ol,1-Aminopropan-2-ol is the organic compound wit...
2,2,2,"1-Chloro-2,4-dinitrobenzene","2,4-Dinitrochlorobenzene (DNCB) is an organic ..."
3,4,4,"1,2-Dichloroethane","The chemical compound 1,2-dichloroethane, comm..."
4,5,5,"1,2,3,5-Tetrahydroxybenzene","1,2,3,5-Tetrahydroxybenzene is a benzenetetrol..."
...,...,...,...,...
7371,7690,8910,Esketamine,"Esketamine, also known as (S)-ketamine or S(+)..."
7372,7691,8912,"3,4-Divanillyltetrahydrofuran","3,4-Divanillyltetrahydrofuran is a lignan foun..."
7373,7692,8913,Aceglutamide,"Aceglutamide (brand name Neuramina), or aceglu..."
7374,7694,8925,Ampelopsin A,Ampelopsin A is a resveratrol dimer found in A...


In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df = pd.read_pickle('drive/MyDrive/wiki_df.pkl')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_pickle('drive/MyDrive/wiki-summarized-1k-known.pkl')
df = df[df['wiki_summarized'] != 0].reset_index()

In [ ]:
df[df['wiki_summarized'] != 0]

,index,cmpdname,wiki,wiki_summarized
0,0,Acetylcarnitine,"Acetyl-L-carnitine, ALCAR or ALC, is an acetyl...","Acetyl L-Carnitine, also known as ALCAR and si..."
1,1,1-Aminopropan-2-ol,1-Aminopropan-2-ol is the organic compound wit...,"The term ""isopropaniolamine"" is often used to ..."
2,2,"1-Chloro-2,4-dinitrobenzene","2,4-Dinitrochlorobenzene (DNCB) is an organic ...",The chapter begins with a brief description of...
3,4,"1,2-Dichloroethane","The chemical compound 1,2-dichloroethane, comm...","The chemical 1,2-dimethylacetonitrile, commonl..."
4,5,"1,2,3,5-Tetrahydroxybenzene","1,2,3,5-Tetrahydroxybenzene is a benzenetetrol...",The article describes the benzene metabolite 1...
...,...,...,...,...
994,1239,Mazaticol,Mazaticol (Pentona) is an anticholinergic used...,This article is about the anticholinergic drug...
995,1240,Mazindol,"Mazindol (brand names Mazanor, Sanorex) is a s...",Mazindo is a drug that has been used as appeti...
996,1241,Edaravone,"Edaravone, sold under the brand name Radicava ...","Edaravarone, sold as Radicavione in Japan and ..."
997,1242,Mebendazole,"Mebendazole (MBZ), sold under the brand name V...",Mebendalz is a drug used to control parasitic ...


In [ ]:
df.to_pickle('drive/MyDrive/benchmark/wiki_summaries_old.pkl')

In [ ]:
titles_new = pd.read_pickle(open('drive/MyDrive/titles_new.pkl', 'rb'))

In [ ]:
titles_new

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_wiki_desc(cmpdname):
  y = ''
  for i in BeautifulSoup(requests.get("https://en.wikipedia.org/wiki/" + cmpdname).content, 'html.parser').find_all('p'):
    y += ''.join(i.get_text().replace("\n", "").replace("{", "").replace("}", "").replace("wikipedia", "").replace('Other reasons this message may be displayed:', ''))

  return y


In [ ]:
titles_1k_sampled = pd.read_pickle('drive/MyDrive/titles_1k_sampled')

In [ ]:
df_1k = pd.DataFrame(columns=['cmpdname', 'wiki'])

In [ ]:
df_1k['cmpdname'] = titles_1k_sampled

In [ ]:
df_1k['wiki'] = df_1k['cmpdname'].apply(get_wiki_desc)

In [ ]:
df_1k = df_1k.reset_index()


In [ ]:
df_sampled = df_1k.copy()
for i in range(len(df_1k)):
  if df_1k['wiki'][i] == '' or "The requested page title contains unsupported" in df_1k['wiki'][i]:
    df_sampled = df_sampled.drop(i)


In [ ]:
df_sampled

,index,cmpdname,wiki
3,18198,Magnesium bicarbonate,Magnesium bicarbonate or magnesium hydrogencar...
12,20131,Lestaurtinib,"Lestaurtinib (rINN, codenamed CEP-701) is a ty..."
25,15112,Fluoroacetaldehyde,Fluoroacetaldehyde is a metabolic precursor of...
50,14589,Methoserpidine,Methoserpidine is an antihypertensive drug rel...
51,31288,Carnosol,Carnosol is a phenolic diterpene found in the ...
...,...,...,...
3975,18874,Nandrolone furylpropionate,Nandrolone furylpropionate (NFP) (JAN) (brand ...
3982,11624,Pipecuronium,Pipecuronium (Arduan) is a bisquaternary amino...
3985,19857,Dehydroalanine,"Dehydroalanine (Cα,β-didehydroalanine, α,β-di-..."
3987,22858,3-deoxy-D-arabino-heptulosonate-7-phosphate,3-Deoxy-D-arabino-heptulosonic acid 7-phosphat...


In [ ]:
df = pd.read_pickle('drive/MyDrive/df_pseudo.pkl')
embeddings = pd.read_pickle('drive/MyDrive/bert_embeddings_full_description')

In [ ]:
indices = df[10000:100000].sample(frac=0.25).index.values

In [ ]:
cmpdnames = list(df['cmpdname'][indices])
cids = list(df['cid'][indices])

In [ ]:
descs = []
good_names = []
good_cids = []

In [ ]:
for i in range(len(cmpdnames)):
  if len(descs) > 1000:
    break

  if i % 100 == 0:
    print(f"{i} / {len(cmpdnames)}, {len(descs)}")
  if '[' not in cmpdnames[i]:
    desc = get_wiki_desc(cmpdnames[i])
    if desc != '':
      descs.append(desc)
      good_names.append(cmpdnames[i])
      good_cids.append(cids[i])


In [ ]:
desc_df = pd.DataFrame(columns=['cid', 'compdname', 'desc'])
desc_df['cid'] = good_cids
desc_df['compdname'] = good_names
desc_df['desc'] = descs


In [ ]:

import time
import pickle
st = time.time()
from IPython.display import clear_output
desc_df['wiki_summarized'] = [None for i in range(len(desc_df))]

for i in range(len(desc_df)):
  time_diff = round((time.time() - st) / 60, 2)
  pct_done = (i+1) / len(desc_df)
  eta = round(time_diff/pct_done, 2)

  if (i+1) % 1000 == 0 and i > 1041:
    pickle.dump(df, open(f'drive/MyDrive/wiki_summarized_save{i}.pkl', 'wb'))
  print(f"ETA: {eta // 60} hours and {eta} minutes, PCT done: {100* round(pct_done, 2)}%, Time gone by: {time_diff // 60} hours and {time_diff} minutes, i: {i}")
  try:
    desc_df['wiki_summarized'][i] = summarize(desc_df['desc'][i])
  except:
    continue
  clear_output()


In [ ]:
desc_df = desc_df.dropna().reset_index()

In [ ]:
desc_df['cid']

0       168464
1      9831659
2      5311309
3      5282096
4       439318
        ...   
987    9915879
988    5359967
989    6327608
990      64735
991    5280789
Name: cid, Length: 992, dtype: int64

In [ ]:
df_pseudo = pd.read_pickle('drive/MyDrive/df_pseudo.pkl')
embeddings = pd.read_pickle('drive/MyDrive/bert_embeddings_full_description')

In [ ]:
iarr = []
for i in range(len(df_pseudo['cid'])):
  if df_pseudo['cid'][i] in desc_df['cid']:
    df_pseudo = df_pseudo.drop(i)
    iarr.append(i)


In [ ]:
embeddings = np.delete(embeddings, iarr, 0)

In [ ]:
df_pseudo = df_pseudo.reset_index()

In [ ]:
embeddings

In [ ]:
pickle.dump(embeddings, open('drive/MyDrive/bert_embeddings_full_description_1k_missing.pkl', 'wb'))

df_pseudo.to_pickle('drive/MyDrive/df_pseudo_1k_missing.pkl')

In [ ]:
len(embeddings)

327726

In [ ]:
len(df_pseudo)

327726

In [ ]:
desc_df.to_pickle('drive/MyDrive/benchmark/wiki_summaries_new.pkl')